In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 49ms/step - loss: 1.3609 - accuracy: 0.5290
Epoch 2/50
8/8 [==============================] - 0s 48ms/step - loss: 1.2928 - accuracy: 0.5433
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2093 - accuracy: 0.5794
Epoch 4/50
8/8 [==============================] - 0s 48ms/step - loss: 1.0815 - accuracy: 0.7218
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 0.9465 - accuracy: 0.8061
Epoch 6/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7500 - accuracy: 0.9025
Epoch 7/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6404 - accuracy: 0.9146
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 0.5879 - accuracy: 0.9299
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5015 - accuracy: 0.9485
Epoch 10/50
8/8 [==============================] - 0s 46ms/step - loss: 0.4325 - accuracy: 0.9595
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 50ms/step - loss: 0.0604 - accuracy: 1.0000
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0572 - accuracy: 1.0000
Epoch 36/50
8/8 [==============================] - 0s 48ms/step - loss: 0.0530 - accuracy: 1.0000
Epoch 37/50
8/8 [==============================] - 0s 48ms/step - loss: 0.0505 - accuracy: 1.0000
Epoch 38/50
8/8 [==============================] - 0s 48ms/step - loss: 0.0475 - accuracy: 1.0000
Epoch 39/50
8/8 [==============================] - 0s 48ms/step - loss: 0.0445 - accuracy: 1.0000
Epoch 40/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0421 - accuracy: 1.0000
Epoch 41/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0399 - accuracy: 1.0000
Epoch 42/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0379 - accuracy: 1.0000
Epoch 43/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0358 - accuracy: 1.0000
Epoch 44/50
8/8 [===============

8/8 [==============================] - 0s 48ms/step - loss: 0.2316 - accuracy: 0.9847
Epoch 18/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1907 - accuracy: 0.9890
Epoch 19/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1935 - accuracy: 0.9803
Epoch 20/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1711 - accuracy: 0.9912
Epoch 21/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1417 - accuracy: 0.9978
Epoch 22/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1312 - accuracy: 0.9945
Epoch 23/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1217 - accuracy: 0.9978
Epoch 24/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1103 - accuracy: 0.9978
Epoch 25/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1014 - accuracy: 1.0000
Epoch 26/50
8/8 [==============================] - 0s 48ms/step - loss: 0.0942 - accuracy: 1.0000
Epoch 27/50
8/8 [===============

8/8 [==============================] - 0s 49ms/step - loss: 0.0230 - accuracy: 1.0000
 >#5: 94.760 
[94.3231463432312, 95.63318490982056, 95.19650936126709, 94.75982785224915, 94.75982785224915]
 Accuracy: 94.934% (+/-0.445) 


In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 1.3335 - accuracy: 0.5761
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2341 - accuracy: 0.6386
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 1.0410 - accuracy: 0.7974
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8457 - accuracy: 0.8839
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7447 - accuracy: 0.9091
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6843 - accuracy: 0.9222
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5936 - accuracy: 0.9441
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 0.4939 - accuracy: 0.9693
Epoch 9/50
8/8 [==============================] - 0s 48ms/step - loss: 0.4460 - accuracy: 0.9748
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3914 - accuracy: 0.9836
Epoch 11/50
8/8 [============

Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0922 - accuracy: 1.0000
Epoch 28/50
8/8 [==============================] - 0s 52ms/step - loss: 0.0849 - accuracy: 1.0000
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0776 - accuracy: 1.0000
Epoch 30/50
8/8 [==============================] - 0s 52ms/step - loss: 0.0725 - accuracy: 1.0000
Epoch 31/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0657 - accuracy: 1.0000
Epoch 32/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0609 - accuracy: 1.0000
Epoch 33/50
8/8 [==============================] - 0s 54ms/step - loss: 0.0561 - accuracy: 1.0000
Epoch 34/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0532 - accuracy: 1.0000
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0475 - accuracy: 1.0000
Epoch 36/50
8/8 [==============================] - 0s 53ms/step - loss: 0.0456 - accuracy: 1.0000
Epoch 37/50
8/8 [===

 >#3: 61.572 
Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 1.3516 - accuracy: 0.5455
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2283 - accuracy: 0.6418
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 1.1039 - accuracy: 0.7262
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9378 - accuracy: 0.8269
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7496 - accuracy: 0.9014
Epoch 6/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6097 - accuracy: 0.9496
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 0.5569 - accuracy: 0.9573
Epoch 8/50
8/8 [==============================] - 0s 51ms/step - loss: 0.4931 - accuracy: 0.9639
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4400 - accuracy: 0.9650
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3902 - accuracy: 0.9748
Epoch 11/50
8/8

8/8 [==============================] - 0s 52ms/step - loss: 0.0623 - accuracy: 1.0000
Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0559 - accuracy: 1.0000
Epoch 28/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0507 - accuracy: 1.0000
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0454 - accuracy: 1.0000
Epoch 30/50
8/8 [==============================] - 0s 54ms/step - loss: 0.0408 - accuracy: 1.0000
Epoch 31/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0367 - accuracy: 1.0000
Epoch 32/50
8/8 [==============================] - 0s 53ms/step - loss: 0.0334 - accuracy: 1.0000
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0302 - accuracy: 1.0000
Epoch 34/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0273 - accuracy: 1.0000
Epoch 35/50
8/8 [==============================] - 0s 53ms/step - loss: 0.0246 - accuracy: 1.0000
Epoch 36/50
8/8 [===============

In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 1.5073 - accuracy: 0.5115
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 1.2660 - accuracy: 0.6407
Epoch 3/50
8/8 [==============================] - 0s 52ms/step - loss: 1.1337 - accuracy: 0.7229
Epoch 4/50
8/8 [==============================] - 0s 51ms/step - loss: 0.9735 - accuracy: 0.8204
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 0.8889 - accuracy: 0.8488
Epoch 6/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7491 - accuracy: 0.9135
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6840 - accuracy: 0.9222
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6137 - accuracy: 0.9365
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 0.5357 - accuracy: 0.9682
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 0.4881 - accuracy: 0.9825
Epoch 11/50
8/8 [============

Epoch 27/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1814 - accuracy: 0.9967
Epoch 28/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1713 - accuracy: 0.9989
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1626 - accuracy: 0.9978
Epoch 30/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1518 - accuracy: 0.9989
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1426 - accuracy: 0.9989
Epoch 32/50
8/8 [==============================] - 0s 52ms/step - loss: 0.1362 - accuracy: 0.9978
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1295 - accuracy: 0.9989
Epoch 34/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1233 - accuracy: 0.9967
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1190 - accuracy: 0.9978
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1101 - accuracy: 0.9978
Epoch 37/50
8/8 [===

 >#3: 62.882 
Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 1.4918 - accuracy: 0.5279
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 1.2944 - accuracy: 0.6068
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2140 - accuracy: 0.6594
Epoch 4/50
8/8 [==============================] - 0s 51ms/step - loss: 1.0639 - accuracy: 0.7514
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 0.9702 - accuracy: 0.8105
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 0.8049 - accuracy: 0.8861
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7546 - accuracy: 0.9014
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6514 - accuracy: 0.9398
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 0.5838 - accuracy: 0.9485
Epoch 10/50
8/8 [==============================] - 0s 52ms/step - loss: 0.5384 - accuracy: 0.9628
Epoch 11/50
8/8

8/8 [==============================] - 0s 50ms/step - loss: 0.1793 - accuracy: 1.0000
Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1698 - accuracy: 1.0000
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1595 - accuracy: 1.0000
Epoch 29/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1493 - accuracy: 1.0000
Epoch 30/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1432 - accuracy: 0.9989
Epoch 31/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1321 - accuracy: 1.0000
Epoch 32/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1250 - accuracy: 1.0000
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1184 - accuracy: 0.9989
Epoch 34/50
8/8 [==============================] - 0s 52ms/step - loss: 0.1099 - accuracy: 1.0000
Epoch 35/50
8/8 [==============================] - 0s 52ms/step - loss: 0.1030 - accuracy: 1.0000
Epoch 36/50
8/8 [===============